In [15]:
import json
import os
import pickle

import pandas as pd
from tqdm import tqdm

from src.embedding_model import EmbeddingModel, EmbeddingTokenizer

tqdm.pandas()

In [18]:
model = EmbeddingModel()
tokenizer = EmbeddingTokenizer()
model.load()
tokenizer.load()

In [16]:
df = pd.read_csv("df_with_split_paragraphs.csv")
df["split_paragraphs"] = df["split_paragraphs"].apply(
    lambda x: "" if pd.isna(x) else x
)  # this is needed because of pandas reading from csv logic

In [20]:
grouped_df = df.groupby("id")
for id, chunk in tqdm(grouped_df):
    # chunk['embeddings'] = chunk['text_column'].apply(lambda text: model(tokenizer([text])))
    if not os.path.isfile(f"./tmp/tmp_save{id}"):
        chunk["split_paragraphs"] = chunk["split_paragraphs"].apply(
            lambda plaintexts: json.dumps(
                [
                    (item[0], item[1], model(tokenizer([item[1]])))
                    for item in json.loads(plaintexts)
                ]
            )
        )

        chunk.to_csv(f"./tmp/tmp_save{id}", index=False)

100%|██████████| 1802/1802 [00:00<00:00, 31134.06it/s]


In [17]:
# From temp
def load_from_save(id):
    return pickle.dumps(
        json.loads(pd.read_csv(f"./tmp/tmp_save{id}")["split_paragraphs"][0])
    )


# split_paragraphs er json string hvor 3 element (index 2) er embeddings til texten (index 1)
df["split_paragraphs"] = df["id"].progress_apply(load_from_save)
df.to_pickle("df_with_embeddings_small.pkl")

  0%|          | 0/1812 [00:00<?, ?it/s]

100%|██████████| 1812/1812 [15:40<00:00,  1.93it/s] 


In [ ]:
df = pd.read_pickle("df_with_embeddings_small.pkl")

In [13]:
print(pickle.loads(df["split_paragraphs"][1])[0])
print(pickle.loads(df["split_paragraphs"][1])[0][2][0][0])

[1, 'RI', [[0.08970171958208084, 0.0030147007200866938, -0.03376699984073639, -0.0750034973025322, 0.09702090173959732, -0.04461755231022835, 0.03382162004709244, 0.05246099457144737, 0.07805035263299942, -0.017218580469489098, 0.07314585894346237, -0.01395176351070404, 0.03321240469813347, -0.007967398501932621, -0.019382890313863754, 0.04092908278107643, 0.0628131777048111, -0.05246221646666527, -0.02684720791876316, -0.05487829074263573, 0.058261938393116, 0.01882416196167469, -0.0515361912548542, 0.04339101165533066, 0.043364547193050385, 0.011581234633922577, -0.02051268331706524, 0.06104749068617821, 0.08070063591003418, -0.06941981613636017, -0.067593514919281, -0.0378277450799942, 0.06810714304447174, -0.08802173286676407, 0.09085053205490112, 0.07263825833797455, -0.08266671001911163, -0.045699592679739, -0.004267154727131128, -0.07362715899944305, -0.03182963281869888, -0.026316337287425995, -0.003646761877462268, 0.08932524919509888, 0.03843095898628235, 0.06172476336359978,